In [2]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
# import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
# import alpaca_trade_api as tradeapi
import hvplot.pandas
import locale
locale.setlocale( locale.LC_ALL, 'en_CA.UTF-8' )
%matplotlib inline
import yfinance as yf

## Create and Analyse Your Desired Portfolio

In [10]:
#Define Global Variables
# Number of simulations
# Benchmark
number_simulations = 10 #1000 # columns
number_records = 20 #30*252# rows

### Prompt User for list of stocks

In [3]:
# User list of stocks
user_portfolio_name = ["MSFT", "AAPL", "AMZN","GOOGL","FB", "V", "JPM"]

#Set benchmark for portfolio
benchmark=["SPY"]

## Import Data

In [4]:
list_of_tickers=benchmark+user_portfolio_name

data = yf.download(
        tickers = list_of_tickers,
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        period = "30y",
        interval = "1d",
        group_by = 'ticker',
        # adjust all OHLC automatically, adjusted for stock splits
        auto_adjust = True,
        prepost = True,
        threads = True,
        proxy = None
    )
data.head()

[*********************100%***********************]  8 of 8 completed


FB                       GOOGL                        ...  \
           Open High Low Close Volume  Open High Low Close Volume  ...   
Date                                                               ...   
1990-05-14  NaN  NaN NaN   NaN    NaN   NaN  NaN NaN   NaN    NaN  ...   
1990-05-15  NaN  NaN NaN   NaN    NaN   NaN  NaN NaN   NaN    NaN  ...   
1990-05-16  NaN  NaN NaN   NaN    NaN   NaN  NaN NaN   NaN    NaN  ...   
1990-05-17  NaN  NaN NaN   NaN    NaN   NaN  NaN NaN   NaN    NaN  ...   
1990-05-18  NaN  NaN NaN   NaN    NaN   NaN  NaN NaN   NaN    NaN  ...   

                 JPM                                         SPY           \
                Open      High       Low     Close   Volume Open High Low   
Date                                                                        
1990-05-14  1.609271  1.655692  1.609271  1.617008  1546500  NaN  NaN NaN   
1990-05-15  1.617008  1.678904  1.609272  1.663430  1902000  NaN  NaN NaN   
1990-05-16  1.647956  1.655693  1.609272  1.640219  1177800  NaN  NaN NaN   
1990-05-17  1.655693  1.663430  1.624746  1.640219   882900  NaN  NaN NaN   
1990-05-18  1.663429  1.671166  1.655692  1.655692    97800  NaN  NaN NaN   

                         
           Close Volume  
Date                     
1990-05-14   NaN    NaN  
1990-05-15   NaN    NaN  
1990-05-16   NaN    NaN  
1990-05-17   NaN    NaN  
1990-05-18   NaN    NaN  

[5 rows x 40 columns]

## Data cleanup

In [6]:
# Historic adjusted close prices
df_yf = data.sort_index()
df_yf.drop(columns=['Open', 'High', 'Low', 'Volume'], level=1, inplace=True)
df_yf = df_yf.droplevel(axis=1, level=1)
df_yf.index = df_yf.index.date
df_yf.dropna(inplace=True)
df_yf.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [7]:
# Isolate cleaned data
user_portfolio_df=df_yf.drop([benchmark[0]], axis =1)
bm_portfolio_df=df_yf[benchmark[0]]

# Calculate daily returns
user_portfolio_returns = user_portfolio_df.pct_change().dropna()
bm_portfolio_returns = bm_portfolio_df.pct_change().dropna()

## Define variables for Benchmark simulation

In [12]:
bm_portfolio_ret_mean = bm_portfolio_returns.mean()
bm_portfolio_ret_std = bm_portfolio_returns.std()
bm_last_price = bm_portfolio_df[-1]

#TODO provide clear message to user, round to two decimal points
#we might want to illustrate what standard deviation means
print(f'Your portfolio average return from SD to ED is:{bm_portfolio_ret_mean}')
print(f'Your portfolio standard deviation from SD to ED is:{bm_portfolio_ret_std}')
print(f'The present value of your benchmark is from SD to ED is:{bm_last_price}')

Your portfolio average return from SD to ED is:0.0005290687362128598
Your portfolio standard deviation from SD to ED is:0.01034786301221672
The present value of your benchmark is from SD to ED is:286.54998779296875


## Run BM Monte Carlo Simulation

In [18]:
# Global variables being used
# Initialize empty DataFrame to hold simulated prices for each simulation
simulated_price_df = pd.DataFrame()
bm_montecarlo_returns = pd.DataFrame()

# Run the simulation of projecting stock prices
for n in range(number_simulations):
    simulated_benchmark_prices = [bm_last_price]
    for i in range(number_records):
        simulated_benchmark_price = simulated_benchmark_prices[-1] * (1 + np.random.normal(bm_portfolio_ret_mean, bm_portfolio_ret_std))
        simulated_benchmark_prices.append(simulated_benchmark_price)
    simulated_price_df["SPY prices"] = pd.Series(simulated_benchmark_prices)
    simulated_daily_returns = simulated_price_df.pct_change()
    b_weights = [1.00] 
    portfolio_daily_returns = simulated_daily_returns.dot(b_weights)
    bm_montecarlo_returns[f'T{n}'] = (1 + portfolio_daily_returns.fillna(0)).cumprod()
bm_montecarlo_returns.head()

,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.002498,1.006703,0.991255,1.000858,1.004093,0.972007,1.016896,1.012314,0.999569,0.996339
2,0.997085,1.010719,0.998874,1.017401,1.012016,0.970118,1.017016,1.011239,1.012939,0.984365
3,1.006693,1.022006,0.991431,1.019693,1.012467,0.982610,1.009503,1.007738,1.020094,0.980334
4,1.014449,1.022835,0.999326,1.035285,1.014360,0.975682,1.011022,1.004782,1.013480,0.972602


In [21]:
bm_montecarlo_returns.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

## Define variables for Portfolio simulation

In [28]:
pf_means = {}
pf_std = {}
pf_last_day = {}

for stock in user_portfolio_name:
    pf_means[stock] = user_portfolio_df.mean()[stock]
    pf_std[stock] = user_portfolio_df.std()[stock]
    pf_last_day[stock] = user_portfolio_df[stock][-1]
#TODO Create a table to display this information for the user
pf_last_day

{'MSFT': 182.50999450683594,
 'AAPL': 311.4100036621094,
 'AMZN': 2356.949951171875,
 'GOOGL': 1375.1800537109375,
 'FB': 210.10000610351562,
 'V': 179.47000122070312,
 'JPM': 86.9000015258789}

In [26]:
#test=Dataframe.from_records([bm_means,bm_std,bm_last_day])

In [35]:
# Global variables being used
# Initialize empty DataFrame to hold simulated prices for each simulation
pf_simulated_price_df = pd.DataFrame()
pf_montecarlo_returns = pd.DataFrame()

# Run the simulation of projecting stock prices
for n in range(number_simulations):
    simulated_portfolio_prices = {stock:[pf_last_day[stock]] for stock in user_portfolio_name}
    for i in range(number_records):
        for stock in user_portfolio_name:
            simulated_stocks_price = simulated_benchmark_prices[-1] * (1 + np.random.normal(pf_means[stock], pf_std[stock]))
            simulated_portfolio_prices[stock].append(simulated_stocks_price)
#     pf_simulated_price_df["SPY prices"] = pd.Series(simulated_benchmark_prices)
    simulated_prices_df = pd.DataFrame(simulated_portfolio_prices)
    simulated_daily_returns = simulated_prices_df.pct_change()
    simulated_daily_returns.dropna(inplace=True)
    #TODO set this to by dynamic
    p_weights = [1/7,1/7,1/7,1/7,1/7,1/7,1/7] 
    portfolio_daily_returns = simulated_daily_returns.dot(p_weights)
    pf_montecarlo_returns[f'T{n}'] = (1 + portfolio_daily_returns.fillna(0)).cumprod()
pf_montecarlo_returns.head()   
#     pf_montecarlo_returns[n] = (1+portfolio_daily_returns.fillna(0)).cumprod()

,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9
1,147.784607,120.683710,120.818669,187.930953,158.521561,124.160119,144.832310,135.991623,126.465555,175.758126
2,184.322139,296.493668,224.643274,143.989549,-22.498989,431.818737,107.793543,92.127371,133.697210,197.877755
3,246.289662,285.887397,-243.496639,581.707552,-26.239273,616.436501,69.143496,300.301884,250.427334,320.786092
4,371.397015,441.423576,-113.725596,504.023731,7.890992,678.465974,66.837128,398.044835,340.833770,266.790164
5,204.570854,822.305956,-91.087194,400.019001,5.281756,1382.640151,61.751486,414.018224,281.594875,434.745831


In [33]:
pf_montecarlo_returns

,0,1,2,3,4,5,6,7,8,9
1,130.735194,150.639238,166.741949,106.134817,147.090022,134.174065,148.556930,203.432792,120.603704,99.480307
2,-312.539895,318.056789,168.351257,149.818784,89.207960,193.114957,235.816783,172.902625,249.678722,162.514492
3,-255.733814,282.409045,187.035517,281.712210,267.761987,179.459134,268.668595,414.560811,224.610150,145.537663
4,-313.860906,441.064114,159.252799,251.753293,590.330670,764.258942,215.491650,337.841316,435.322730,161.319585
5,-149.115383,609.857477,466.340452,621.181859,831.343427,3154.866400,363.825918,265.056001,393.277658,709.863744
6,-99.798082,739.833860,465.893510,778.642870,905.149330,1291.939161,392.178983,284.336523,359.542800,485.152244
7,-106.861496,2913.693754,390.807422,1502.815105,421.858538,1466.267778,459.370047,412.929116,820.018658,733.528411
8,-103.358418,-1101.295412,89.825440,4307.572680,1315.216099,8672.280777,543.225682,457.590028,813.484021,739.035931
9,-259.111881,-1365.711977,441.199774,-671.708932,691.029449,9432.399657,551.502038,401.399982,872.028268,1577.163688
10,-138.856330,-1294.136802,395.995327,18.049717,-1238.122894,9114.052099,1229.101272,360.242667,1033.132455,1608.539215
